# Project Setup for Azure AI Foundry

This notebook guides you through setting up an AI project in Azure AI Foundry. You'll learn:
1. Creating a new AI project
2. Configuring project settings
3. Managing project resources
4. Best practices for project organization

## Prerequisites
- Completed authentication setup from previous notebook
- Azure AI Foundry access
- Required Python packages installed

In [ ]:
# Set up environment variables for local testing
import os

# Check for required environment variables
required_vars = {
    "AZURE_SUBSCRIPTION_ID": os.getenv("AZURE_SUBSCRIPTION_ID"),
    "AZURE_RESOURCE_GROUP": os.getenv("AZURE_RESOURCE_GROUP"),
    "AZURE_TENANT_ID": os.getenv("AZURE_TENANT_ID"),
    "AZURE_CLIENT_ID": os.getenv("AZURE_CLIENT_ID"),
    "AZURE_CLIENT_SECRET": os.getenv("AZURE_CLIENT_SECRET")
}

# Check which variables are missing
missing_vars = [var for var, value in required_vars.items() if not value]

if missing_vars:
    print("× Missing required environment variables:")
    for var in missing_vars:
        print(f"  - {var}")
    print("\nPlease set these variables before running the notebook:")
    print("```bash")
    print("export AZURE_SUBSCRIPTION_ID='your-subscription-id'")
    print("export AZURE_RESOURCE_GROUP='your-resource-group'")
    print("export AZURE_TENANT_ID='your-tenant-id'")
    print("export AZURE_CLIENT_ID='your-client-id'")
    print("export AZURE_CLIENT_SECRET='your-client-secret'")
    print("```")
else:
    print("✓ All required environment variables are set")

In [ ]:
# Import required libraries
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.inference import InferenceClient, ChatCompletionsClient
from azure.ai.evaluation import EvaluationClient, TextEvaluator
from azure.ai.contentsafety import ContentSafetyClient
import azure.monitor.opentelemetry._autoinstrument
import os
import json

# Initialize credentials
try:
    credential = DefaultAzureCredential()
    print("✓ Successfully initialized DefaultAzureCredential")
except Exception as e:
    print(f"× Error initializing credentials: {str(e)}")

## Initialize AI Project Client
First, let's create our client instance using the credentials we set up:

In [ ]:
# Create AI Project client
try:
    client = AIProjectClient(
        subscription_id=os.getenv("AZURE_SUBSCRIPTION_ID"),
        resource_group=os.getenv("AZURE_RESOURCE_GROUP"),
        credential=credential
    )
    print("✓ Successfully initialized AIProjectClient")
except Exception as e:
    print(f"× Error initializing client: {str(e)}")

## Create a New Project
Let's create a customer service AI project:

In [ ]:
def create_project(name="customer-service-agent", description="AI-powered customer service agent"):
    """Create a new AI Foundry project."""
    try:
        project = client.projects.create(
            name=name,
            description=description,
            tags={
                "environment": "development",
                "purpose": "customer-service",
                "workshop": "true"
            }
        )
        print(f"✓ Successfully created project: {project.name}")
        return project
    except Exception as e:
        print(f"× Error creating project: {str(e)}")
        return None

# Create the project
project = create_project()

## Project Configuration
Configure project settings and resources:

In [ ]:
def configure_project(project):
    """Configure project settings."""
    if not project:
        print("× No project to configure")
        return
    
    try:
        # Update project settings
        project = client.projects.update(
            project_name=project.name,
            settings={
                "deployment": {
                    "environment": "development",
                    "region": "eastus",
                    "sku": "standard"
                },
                "monitoring": {
                    "metrics_enabled": True,
                    "logging_level": "INFO"
                }
            }
        )
        print("✓ Successfully configured project settings")
    except Exception as e:
        print(f"× Error configuring project: {str(e)}")

# Configure the project
configure_project(project)

## List Project Resources
View the resources associated with your project:

In [ ]:
def list_project_resources(project_name):
    """List resources in the project."""
    try:
        resources = client.projects.list_resources(project_name=project_name)
        print("
Project Resources:")
        for resource in resources:
            print(f"- {resource.name} ({resource.type})")
    except Exception as e:
        print(f"× Error listing resources: {str(e)}")

# List resources if project was created
if project:
    list_project_resources(project.name)

## Project Organization Best Practices

1. **Naming Conventions**
   - Use descriptive, lowercase names
   - Include environment indicator
   - Add purpose/function identifier

2. **Resource Management**
   - Group related resources
   - Use consistent tagging
   - Regular cleanup of unused resources

3. **Access Control**
   - Implement role-based access
   - Regular permission reviews
   - Audit access logs

In [ ]:
def apply_best_practices(project):
    """Apply best practices to project."""
    if not project:
        print("× No project to configure")
        return
    
    try:
        # Update project with best practice configurations
        project = client.projects.update(
            project_name=project.name,
            tags={
                "owner": "workshop-participant",
                "cost-center": "training",
                "environment": "development",
                "purpose": "customer-service",
                "created-date": "2024-01",
                "cleanup-after": "2024-02"
            }
        )
        print("✓ Successfully applied best practices")
        
        # Display project details
        print("
Project Configuration:")
        print(f"Name: {project.name}")
        print(f"Description: {project.description}")
        print("Tags:")
        for key, value in project.tags.items():
            print(f"  {key}: {value}")
            
    except Exception as e:
        print(f"× Error applying best practices: {str(e)}")

# Apply best practices
if project:
    apply_best_practices(project)

## Cleanup (Optional)
If you want to remove the project:

In [ ]:
def cleanup_project(project_name):
    """Delete the project and associated resources."""
    try:
        client.projects.delete(project_name=project_name)
        print(f"✓ Successfully deleted project: {project_name}")
    except Exception as e:
        print(f"× Error deleting project: {str(e)}")

# Uncomment to cleanup (careful!)
# if project:
#     cleanup_project(project.name)

## Next Steps
- Deploy models to your project
- Configure endpoints
- Set up monitoring
- Implement the customer service agent

Remember to clean up resources if you're done with the workshop!